In [32]:
import pandas as pd
import numpy as np

df_fire_conf = pd.read_csv('../fire_confidence/fire_confidence_clean.csv')
df_fire_news = pd.read_csv('../fire_news/fire_news_clean.csv', dtype={'time': str})

#print(df_fire_conf.head())
#print(df_fire_news.head())



In [40]:
from datetime import datetime

# Convert dates to datetime objects to ensure proper comparison
df_fire_conf['date'] = pd.to_datetime(df_fire_conf['date'])
df_fire_news['date'] = pd.to_datetime(df_fire_news['date'])


# Group data by date in both dataframes
grouped_conf = df_fire_conf.groupby('date')
grouped_news = df_fire_news.groupby('date')

new_locations  = []
#Iterate over each group in df_fire_news
for date, group_news in grouped_news:
    print(date)
    # Check if the date exists in df_fire_conf
    if date in grouped_conf.groups:
        print("Matching fires from confidence dataset:")
        group_conf = grouped_conf.get_group(date)
        # print(group_conf)
        # print(group_news)
        # Iterate over each row in group_news and compare with group_conf
        for index_news, row_news in group_news.iterrows():
            # Compare with all rows in group_conf
            match_found = False
            for index_conf, row_conf in group_conf.iterrows():
                # Calculate absolute differences
                lat_diff = abs(row_news['latitude'] - row_conf['latitude'])
                lon_diff = abs(row_news['longitude'] - row_conf['longitude'])

                # Check if differences between points exceed 0.1 degree
                if lat_diff <= 0.1 and lon_diff <= 0.1:
                    match_found = True
                    break

            # If no match found, consider it a new location
            if not match_found:
                new_locations.append({
                    'latitude': row_news['latitude'],
                    'longitude': row_news['longitude'],
                    'date': date,
                    'time': row_news['time']
                })
    else:
        print("No matching fires found in confidence dataset.")
        # print(group_news)

    print("-------------------------------------")


df_new_locations = pd.DataFrame(new_locations)


display(df_new_locations)

#print(len(df_new_locations))

df_new_locations.to_csv('new_locations_from_firenews.csv', index=False)


2023-04-28 00:00:00
No matching fires found in confidence dataset.
-------------------------------------
2023-05-11 00:00:00
No matching fires found in confidence dataset.
-------------------------------------
2023-05-22 00:00:00
No matching fires found in confidence dataset.
-------------------------------------
2023-06-22 00:00:00
Matching fires from confidence dataset:
-------------------------------------
2023-07-04 00:00:00
No matching fires found in confidence dataset.
-------------------------------------
2023-07-14 00:00:00
Matching fires from confidence dataset:
-------------------------------------
2023-07-17 00:00:00
Matching fires from confidence dataset:
-------------------------------------
2023-07-18 00:00:00
Matching fires from confidence dataset:
-------------------------------------
2023-07-19 00:00:00
Matching fires from confidence dataset:
-------------------------------------
2023-07-20 00:00:00
Matching fires from confidence dataset:
------------------------------

,latitude,longitude,date,time
0,39.55,8.78,2023-06-22,0902
1,41.72,15.84,2023-07-14,0943
2,39.99,18.39,2023-07-17,1030
3,37.96,12.97,2023-07-18,1044
4,40.06,18.01,2023-07-19,1117
...,...,...,...,...
60,36.79,14.71,2024-04-30,0858
61,39.75,8.60,2024-04-30,0944
62,40.71,13.89,2024-05-22,1003
63,38.09,15.46,2024-05-23,0840


In [45]:
df_fire_conf = pd.read_csv('../fire_confidence/fire_confidence_clean.csv')
df_new_locations_from_firenews = pd.read_csv('../fire_labels/new_locations_from_firenews.csv', dtype={'time': str})

columns_to_append = ['latitude_clean', 'longitude_clean', 'date', 'time']
df_to_append = df_fire_conf[columns_to_append]

# Rename columns in df_to_append to match df_new_locations_from_firenews
df_to_append.columns = ['latitude', 'longitude', 'date', 'time']

df_combined = pd.concat([df_new_locations_from_firenews, df_to_append], ignore_index=True)

#confirm merge
if len(df_combined) == len(df_new_locations_from_firenews) + len(df_fire_conf):
    print("merge successful")
else:
    print("merge failed")

# Example of saving to CSV
df_combined.to_csv('fire_labels_final.csv', index=False)

65
1082
1147
